In [32]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### Murder Mystery Revised

Oh no! Someone died, and we have no idea who did it!

There are three people who could have done it:
- The gardener
- The maid
- The valet

And there's 3 potential weapons that could have been used:
- The shovel
- The feather duster
- The car

The following are our priors for the three suspects, in order:

In [33]:
suspects = [ 'gardener', 'maid', 'valet' ]
prior_probs = [ 0.4, 0.2, 0.4 ]

These are our priors, because a maid seems generally less likely to be a murderer than anyone else.

Now, we have the likelihoods of each of the suspects using each of the plausible weapons, in order:

In [34]:
weapons = [ 'shovel', 'feather_duster', 'car' ]
suspect_probs =   { 
    'gardener': [ 0.6, 0.2, 0.2 ],
    'maid':     [ 0.2, 0.6, 0.2 ],
    'valet':    [ 0.2, 0.2, 0.6 ]
}

These are our likelihoods because each weapon is more likely to be used by someone who uses it as part of their profession.

Below we approximate the joint probabilities using a simulation.

In [35]:
def draw_prior(suspects, probs):
    return np.random.choice(suspects, p=probs)

def draw_model(weapons, suspect, suspect_probs):
    return np.random.choice(weapons, p=suspect_probs[suspect])

def draw_joint(weapons, suspects, prior_probs, suspect_probs):
    s = draw_prior(suspects, prior_probs)
    w = draw_model(weapons, s, suspect_probs)
    return (s, w)

def simulator(*args, num_sims=10000):
    return [draw_joint(*args) for _ in range(num_sims)]


num_sims = 10_000
sims = simulator(weapons, suspects, prior_probs, suspect_probs, num_sims=num_sims)


scenarios, counts = np.unique([f"{s}{w}" for s, w in sims], return_counts=True)
counts_dict = dict(zip(scenarios, counts))

approximate_joint_probs = np.zeros((len(suspects), len(weapons)))
for i in range(len(suspects)):
    for j in range(len(weapons)):
        count = counts_dict[f"{suspects[i]}{weapons[j]}"]
        approximate_joint_probs[i][j] = round(count / num_sims, 2)

Then, we calculate the joint probablities analytically.

In [36]:
analytic_joint_probs = np.zeros((len(suspects), len(weapons)))
for i in range(len(suspects)):
    s = suspects[i]
    for j in range(len(weapons)):
        analytic_joint_probs[i][j] =  suspect_probs[s][j] * prior_probs[i]

Then, we compare the joint probabilites found by both methods.

In [37]:
df = pd.DataFrame(approximate_joint_probs, columns=weapons, index=suspects)
print("Approximate Joint Probabilities:")
print(df, "\n")

df = pd.DataFrame(analytic_joint_probs, columns=weapons, index=suspects)
print("Analytic Joint Probabilities:")
print(df, "\n")

diff = np.round(approximate_joint_probs - analytic_joint_probs, 2)
df = pd.DataFrame(diff, columns=weapons, index=suspects)
print("Difference:")
print(df, "\n")

Approximate Joint Probabilities:
          shovel  feather_duster   car
gardener    0.24            0.08  0.08
maid        0.04            0.12  0.04
valet       0.08            0.08  0.24 

Analytic Joint Probabilities:
          shovel  feather_duster   car
gardener    0.24            0.08  0.08
maid        0.04            0.12  0.04
valet       0.08            0.08  0.24 

Difference:
          shovel  feather_duster  car
gardener     0.0            -0.0 -0.0
maid        -0.0             0.0 -0.0
valet       -0.0            -0.0  0.0 



The number of simulations needs to be around 10,000 for the approximation to be indistinguishable from the analytic ones.